In [ ]:
import face_recognition
from PIL import Image
import cv2
import numpy as np
import paramiko
import pymysql
import time
import io
# from flask import Flask, Response, render_template

# This is a super simple (but slow) example of running face recognition on live video from your webcam.
# There's a second example that's a little more complicated but runs faster.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.


# First, automatically enable SSH and start the camera
# SSH connection information
hostname = '192.168.137.32'  # Raspberry Pi's IP address
port = 22  # SSH port number, default is 22
username = 'wwb2'  # Raspberry Pi's username
password = 'pi'  # Raspberry Pi's user password

# Create an SSH client object
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    # Connect to the Raspberry Pi
    client.connect(hostname, port, username, password)
    
    # Execute the "sudo motion" command
    stdin, stdout, stderr = client.exec_command('sudo motion')

    # Get the command execution result
    output = stdout.read().decode('utf-8')
    print(output)
    
    # Close the SSH connection
    client.close()

except paramiko.AuthenticationException:
    print("Authentication failed. Please check the username and password.")

except paramiko.SSHException as ssh_err:
    print("SSH connection error:", ssh_err)

except paramiko.Exception as e:
    print("Error:", e)

    
# Database connection configuration
config = {
    'host': '192.168.137.203',
    'user': 'root',
    'passwd': 'tansome',
    'db': 'fatigueTest',
    'charset': 'utf8mb4',
}

# Connect to MySQL database
connection = pymysql.connect(**config)

try:
    with connection.cursor() as cursor:
        # Clear the table before starting
        clear_query = "TRUNCATE TABLE facial"
        cursor.execute(clear_query)
except pymysql.Error as e:
    print("Error while connecting to MySQL:", e)

# Define the URL of the video source
url = "http://192.168.137.32:8081/"  # Raspberry Pi 1, used for facial recognition

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(url)

# Load sample pictures and learn how to recognize them.
lwz_image = Image.open("lwz.jpg")
lwz_image = lwz_image.convert("RGB")
lwz_face_encoding = face_recognition.face_encodings(np.array(lwz_image))[0]

wwb_image = Image.open("wwb.jpg")
wwb_image = wwb_image.convert("RGB")
wwb_face_encoding = face_recognition.face_encodings(np.array(wwb_image))[0]

zjy_image = Image.open("zjy.jpg")
zjy_image = zjy_image.convert("RGB")
zjy_face_encoding = face_recognition.face_encodings(np.array(zjy_image))[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    lwz_face_encoding,
    wwb_face_encoding,
    zjy_face_encoding
]
known_face_names = [
    "LIN WUZHEN",
    "WANG WEIBO",
    "ZHANG JINGYUAN"
]

# Set the counter value
frame_count = 0
# Set the interval to process face recognition
interval = 5

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    frame = cv2.flip(frame, 0)
    frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    # Increment the frame counter
    frame_count += 1
    
    if frame_count % interval == 0:
        
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_frame = frame[:, :, ::-1]

        # Find all the faces and face encodings in the frame of video
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        # Loop through each face in this frame of video
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.5)

            name = "Unknown"

            # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (225, 0, 0), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 17), (right, bottom), (255 , 0, 0), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)
            

        # Display the resulting image
        cv2.imshow('Video', frame)
        
        # Check if any face is detected
        if len(face_encodings) > 0:
            with connection.cursor() as cursor:
                # Convert the frame to PIL Image
                pil_image = Image.fromarray(frame)

                # Create a file-like object to hold the image data
                image_stream = io.BytesIO()

                # Save the PIL Image to the file-like object as JPEG
                pil_image.save(image_stream, format='JPEG')

                # Rewind the image stream
                image_stream.seek(0)

                # Read the image data from the stream
                image_data = image_stream.read()

                # Insert data into the database
                insert_query = "INSERT INTO facial (time, video, name) VALUES (%s, %s, %s)"
                data = (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), image_data, name)
                cursor.execute(insert_query, data)
                connection.commit()
        
        # Press 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            # Close the connection
            connection.close()
            break
      
            
# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()


D:\Anaconda\envs\AIOT\lib\site-packages\paramiko\transport.py:32: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  from cryptography.hazmat.backends import default_backend
